In [ ]:
import os
import httplib2
import ConfigParser
import numbers
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
# This class allows to interact with GDrive
# To do it, a service account is require
# https://developers.google.com/identity/protocols/OAuth2ServiceAccount#creatinganaccount
class GDrive(object):
    def __init__(self, oauth_file):

        # Create OAuth credentials
        self._credentials = ServiceAccountCredentials.from_json_keyfile_name(oauth_file,
                                                                             ['https://www.googleapis.com/auth/drive'])
        self._credentials.authorize(httplib2.Http())
        self._gauth = GoogleAuth()
        self._gauth.credentials = self._credentials

    def version(self, resource_id, path, file_handle):
        # Creates object for interacting with GDrive API
        drive = GoogleDrive(self._gauth)

        # Gets remote file and update mime type if necessary
        file1 = drive.CreateFile({'id': resource_id})
        # Set mime type if different
        if file1['mimeType'] != file_handle:
            file1['mimeType'] = file_handle
        # Replaces contents with local file
        file1.SetContentFile(path)
        file1.Upload()

    def download(self, resource_id, file_name, path):
        # Creates object for interacting with GDrive API
        drive = GoogleDrive(self._gauth)

        # Downloads existing file
        file1 = drive.CreateFile({'id': resource_id})
        file1.GetContentFile(os.path.join(path, file_name))

        return file1

In [ ]:
config = ConfigParser.RawConfigParser()
config.read("datasets.conf")

In [ ]:
# Downloads last MRR matrix
gdrive = GDrive(config.get('gdrive', 'auth_file'))
f = gdrive.download(config.get('gdrive', 'input_csv_file_id'),
                    config.get('mrr_input', 'file_name'),
                    config.get('mrr_input', 'file_dir'))

As we are accessing GDrive with a service account, remote file needs to be shared with it.

In [ ]:
# Loads matrix to pandas and sets column names
input_file = os.path.join(config.get('mrr_input', 'file_dir'), config.get('mrr_input', 'file_name'))
customers_df = pd.read_csv(input_file, sep=',', header=0)

In [ ]:
customers_df

In [ ]:
# Removes all invoice columns but last two completed periods (month-1, month-2)
today = datetime.now().date()
current_period = today.replace(day=1) - relativedelta(months=1)
prior_period = today.replace(day=1) - relativedelta(months=2)

# Removes non-used periods
columns_to_remove = list(customers_df.columns.values)
required_columns = ('account_id', 'name', 'email', 'created_at', 'current_plan',
                    'plan_ends_at', str(current_period), str(prior_period))
for c in required_columns:
    columns_to_remove.remove(c)

# Keeps only last two complete invoicing periods
customers_df.drop(columns_to_remove, axis=1, inplace=True)

# Rename invoicing period columns
customers_df.rename(columns={str(prior_period): 'prior_invoiced', 
                             str(current_period): 'current_invoiced'}, 
                    inplace=True)

In [ ]:
# Cleans na values
customers_df.loc[:, 'prior_invoiced'].fillna(0, inplace=True)
customers_df.loc[:, 'current_invoiced'].fillna(0, inplace=True)

We complete the dataset with two new attributes:
- behaviour: new, churn, upgrade or downgrade
- invoiced variation: invoice variation from prior month to current

In [ ]:
# Enrich data
def is_valid_invoice(value):
    return isinstance(value, numbers.Real) and value > 0.0
    
def get_customer_behaviour(invoices):
    # input: (prior_invoiced, current_invoiced)
    # output: behaviour, invoice variation
    prior = invoices[0] if is_valid_invoice(invoices[0]) else 0.0
    current = invoices[1] if is_valid_invoice(invoices[1]) else 0.0

    if prior == 0.0 and current > 0.0:
        return 'New', round(current, 2)
    if prior > 0.0 and current == 0.0:
        return 'Churn', round(prior * -1, 2)
    if prior < current:
        return 'Upgrade', round(current - prior, 2)
    elif prior > current:
        return 'Downgrade', round(current - prior, 2)
    return 'No change', 0.00

customers_df[['behaviour', 'invoiced_var']] = customers_df[['prior_invoiced', 'current_invoiced']] \
                                              .apply(lambda x: pd.Series(get_customer_behaviour(x)), axis=1)

In [ ]:
def is_included(invoices):
    # input: (prior_invoiced, current_invoiced)
    return invoices[1] > 0 or invoices[0] > 0

# Remove non-active accounts (those with no payments in current or prior month)
included = customers_df[['prior_invoiced', 'current_invoiced']].apply(is_included, axis=1).values
customers_df = customers_df[included]

In [ ]:
customers_df

In [ ]:
# Export to csv
output_file = os.path.join(config.get('output', 'local_file_dir'),
                           config.get('output', 'local_file_name'))
customers_df.to_csv(output_file, encoding='utf-8', index=False)

This example requires the file already exists in GDrive so we uploaded just an empty file in advance via web UI and kept the file ID. 
The reason is, as we are accesing GDrive using a service account, if we create a new file with this account, it will be created in GDrive

In [ ]:
# Updates new version of file to GDrive
gdrive = GDrive(config.get('gdrive', 'auth_file'))
gdrive.version(config.get('gdrive', 'output_csv_file_id'),
               output_file,
               'text/csv')

That's it. Now new data is available in GDrive and can be imported to CartoDB!